In [1]:
import numpy as np 
import pandas as pd
dataset = pd.read_csv('./train.csv') #用來當training資料的
X_test = pd.read_csv('./test.csv')

## Read Data

In [2]:
dataset.tail(5)

,Date,Open Price,Close Price,High Price,Low Price,Volume
2259,22-Dec-2017,2684.22,2683.34,2685.35,2678.13,1383888512
2260,26-Dec-2017,2679.09,2680.50,2682.74,2677.96,1103808384
2261,27-Dec-2017,2682.10,2682.62,2685.64,2678.91,1149108352
2262,28-Dec-2017,2686.10,2687.54,2687.66,2682.69,1126089856
2263,29-Dec-2017,2689.15,2673.61,2692.12,2673.61,1332374016


## train 資料處理
#### 增加movement表示漲跌
#### 由於資料太少，增加high price, low price差為一個特徵

In [3]:
dataset['sub'] = 0.0
for i in range(0, len(dataset.index)):
    dataset['sub'][i] = dataset['High Price'][i] - dataset['Low Price'][i]

dataset['movement'] = 0
for i in range(1, len(dataset.index)):
    if(dataset['Close Price'][i] > dataset['Close Price'][i-1]):
        dataset['movement'][i] = 1

/home/yuen/.local/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/yuen/.local/lib/python3.5/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


由於要預測「明天」是否漲跌，所以將資料向上平移一格。
並且因為最後一天的資料沒有「明天」的漲跌，顧刪除。

In [4]:
# 平移
for i in range(1, len(dataset.index)):
    dataset['movement'][i-1] = dataset['movement'][i]
    
# 刪除最後一天
dataset.drop([len(dataset)-1],inplace=True)
print(dataset.tail(5))

/home/yuen/.local/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


             Date  Open Price  Close Price  High Price  Low Price      Volume  \
2258  21-Dec-2017     2683.02      2684.57     2692.64    2682.40  1933795072   
2259  22-Dec-2017     2684.22      2683.34     2685.35    2678.13  1383888512   
2260  26-Dec-2017     2679.09      2680.50     2682.74    2677.96  1103808384   
2261  27-Dec-2017     2682.10      2682.62     2685.64    2678.91  1149108352   
2262  28-Dec-2017     2686.10      2687.54     2687.66    2682.69  1126089856   

        sub  movement  
2258  10.24         0  
2259   7.22         0  
2260   4.78         1  
2261   6.73         1  
2262   4.97         0  


定義train data

In [5]:
X_train = dataset.iloc[:, 1:7].values # x捨棄date , .value的話會捨棄掉Close Price那些

Y_train = dataset.iloc[:, 7].values # y是movement

In [6]:
print(X_train[0])
print(Y_train[0])

[9.02990000e+02 9.31800000e+02 9.34730000e+02 8.99350000e+02
 4.04827008e+09 3.53800000e+01]
0


## test data 處理

In [7]:
X_test['sub'] = 0.0
for i in range(0, len(X_test.index)):
    X_test['sub'][i] = X_test['High Price'][i] - X_test['Low Price'][i]

# 計算漲跌
X_test['movement'] = 0
for i in range(1, len(X_test.index)):
    if(X_test['Close Price'][i] > X_test['Close Price'][i-1]):
        X_test['movement'][i] = 1
        
# 平移
for i in range(1, len(X_test.index)):
    X_test['movement'][i-1] = X_test['movement'][i]
    
# 刪除最後一天
X_test.drop([len(X_test)-1],inplace=True)
print(X_test.tail(5))

# 定義test資料
x_test = X_test.iloc[:, 1:7].values # x捨棄date , .value的話會捨棄掉Close Price那些
y_test = X_test.iloc[:, 7].values # y是movement

/home/yuen/.local/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/yuen/.local/lib/python3.5/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/yuen/.local/lib/python3.5/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


            Date  Open Price  Close Price  High Price  Low Price      Volume  \
246  21-Dec-2018     2465.38      2416.62     2504.41    2408.55  4560164352   
247  24-Dec-2018     2400.56      2351.10     2410.34    2351.10  1662758784   
248  26-Dec-2018     2363.12      2467.70     2467.76    2346.58  2611875072   
249  27-Dec-2018     2442.50      2488.83     2489.10    2397.94  2386466304   
250  28-Dec-2018     2498.77      2485.74     2520.27    2472.89  2080726656   

        sub  movement  
246   95.86         0  
247   59.24         1  
248  121.18         1  
249   91.16         0  
250   47.38         1  


## SVM

In [8]:
from sklearn import svm, preprocessing

# sigmoid
svm_model_s = svm.SVC(C = 0.05, kernel='sigmoid', probability=True)
svm_model_s.fit(X_train, Y_train)
probability = svm_model_s.predict_proba(x_test)
print(probability[:20])
print(svm_model_s.score(x_test, y_test))

[[0.45918721 0.54081279]
 [0.45957526 0.54042474]
 [0.459584   0.540416  ]
 [0.45933946 0.54066054]
 [0.45929252 0.54070748]
 [0.45940711 0.54059289]
 [0.45953345 0.54046655]
 [0.45944088 0.54055912]
 [0.45959022 0.54040978]
 [0.45949606 0.54050394]
 [0.45963108 0.54036892]
 [0.45964326 0.54035674]
 [0.45962385 0.54037615]
 [0.4595731  0.5404269 ]
 [0.45961854 0.54038146]
 [0.45961522 0.54038478]
 [0.45964967 0.54035033]
 [0.45962356 0.54037644]
 [0.45957149 0.54042851]
 [0.45963837 0.54036163]]
0.5219123505976095


## NN

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(6, 50) 
        self.fc2 = nn.Linear(50, 2)
        
    def forward(self, x):
        x = self.fc1(x)
        x = F.dropout(x, p=0.1)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.sigmoid(x)
        
        return x
    
net = Net()

In [10]:
batch_size = 50 #一組50（一次訓練訓練個2262/50次）
num_epochs = 50 #訓練50次
learning_rate = 0.01
batch_no = len(X_train) # 801

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

In [12]:
from sklearn.utils import shuffle
from torch.autograd import Variable

for epoch in range(num_epochs):
    if epoch % 5 == 0:
        print('Epoch {}'.format(epoch+1))
    X_train, Y_train = shuffle(X_train, Y_train) # 每次都shuffle一次
    # 每個 batch learning
    for i in range(batch_no // batch_size):
        start = i * batch_size
        end = start + batch_size
        #丟給model必須是Variable  才能backward
        x_var = Variable(torch.FloatTensor(X_train[start:end]))#float
        y_var = Variable(torch.LongTensor(Y_train[start:end]))#int
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        ypred_var = net(x_var) 
        loss =criterion(ypred_var, y_var)
        loss.backward()
        optimizer.step()

Epoch 1


/home/yuen/.local/lib/python3.5/site-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch 6
Epoch 11
Epoch 16
Epoch 21
Epoch 26
Epoch 31
Epoch 36
Epoch 41
Epoch 46


In [13]:
test_var = Variable(torch.FloatTensor(x_test), requires_grad=True)
#test_y = Variable(torch.LongTensor(y_val), requires_grad=True)
with torch.no_grad(): #深度學習模型在訓練時會自動計算梯度，若於分析模型在目標函數的表現時不想花多餘資源計算梯度可以使用 with to.no_grad():
    result = net(test_var)
    
values, labels = torch.max(result, 1) # 0以行取max / 0以列取max
#MAX會回傳誰比較大 跟該index

num_right = np.sum(labels.data.numpy() == y_test)
#ACC = 猜中幾個
print('Accuracy {:.2f}'.format(num_right / len(y_test)))

Accuracy 0.52


##  RL

In [14]:
import torch.nn.functional as func

class logit_model(torch.nn.Module):

    def __init__(self):
        super(logit_model, self).__init__()
        self.linear = torch.nn.Linear(6, 2)

    def forward(self, x):
        return(self.linear(x))


net = logit_model()

In [15]:
for epoch in range(num_epochs):
    if epoch % 5 == 0:
        print('Epoch {}'.format(epoch+1))
    X_train, Y_train = shuffle(X_train, Y_train) # 每次都shuffle一次
    # 每個 batch learning
    for i in range(batch_no // batch_size):
        start = i * batch_size
        end = start + batch_size
        #丟給model必須是Variable  才能backward
        x_var = Variable(torch.FloatTensor(X_train[start:end]))#float
        y_var = Variable(torch.LongTensor(Y_train[start:end]))#int
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        ypred_var = net(x_var) 
        loss =criterion(ypred_var, y_var)
        loss.backward()
        optimizer.step()

Epoch 1
Epoch 6
Epoch 11
Epoch 16
Epoch 21
Epoch 26
Epoch 31
Epoch 36
Epoch 41
Epoch 46


In [16]:
test_var = Variable(torch.FloatTensor(x_test), requires_grad=True)
#test_y = Variable(torch.LongTensor(y_val), requires_grad=True)
with torch.no_grad(): #深度學習模型在訓練時會自動計算梯度，若於分析模型在目標函數的表現時不想花多餘資源計算梯度可以使用 with to.no_grad():
    result = net(test_var)

values, labels = torch.max(result, 1) # 0以行取max / 0以列取max

#MAX會回傳誰比較大 跟該index
num_right = np.sum(labels.data.numpy() == y_test)
#ACC = 猜中幾個
print('Accuracy {:.8f}'.format(num_right / len(y_test)))

Accuracy 0.47808765


## improve - 試試crossvalidation

In [17]:
X_train = dataset.iloc[:, 1:7].values # x捨棄date , .value的話會捨棄掉Close Price那些

Y_train = dataset.iloc[:, 7].values # y是movement

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(6, 50) 
        self.fc2 = nn.Linear(50, 2)
        
    def forward(self, x):
        x = self.fc1(x)
        x = F.dropout(x, p=0.1)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.sigmoid(x)
        
        return x
    
net = Net()

In [19]:
batch_size = 50 # 一組50（一次訓練訓練個 TRAIN DATA / 50次）
num_epochs = 50 # 訓練50次
learning_rate = 0.01
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

In [20]:
from sklearn.utils import shuffle
from torch.autograd import Variable
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

train_acc_list = []                                   # 儲存每次訓練模型的準確度
valid_acc_list = []                                   # 儲存每次驗證模型的準確度

# 切train val
kf = KFold(n_splits=5, shuffle=True)
kf.get_n_splits(X_train)#因為x y數量一樣，所以把一個拿進去切就好

for train_index, test_index in kf.split(X_train):

    x_train, x_test = X_train[train_index], X_train[test_index]
    y_train, y_test = Y_train[train_index], Y_train[test_index]
    batch_no = len(x_train) // batch_size
    for epoch in range(num_epochs):
        if epoch % 5 == 0:
            print('Epoch {}'.format(epoch+1))

        # 每個 batch learning
        for i in range(batch_no):
            #print('batch no:', i)
            start = i * batch_size
            end = start + batch_size
            
            net.train()# 進入非訓練模式
            
            #丟給model必須是Variable  才能backward
            x_var = torch.FloatTensor(x_train[start:end])#float
            y_var = torch.LongTensor(y_train[start:end])#int

            # Forward + Backward + Optimize
            optimizer.zero_grad()
            ypred_var = net(x_var) 
            loss =criterion(ypred_var, y_var)
            loss.backward()
            optimizer.step()
            
            net.eval() # 進入非訓練模式
            with torch.no_grad():
                
                x_train = torch.FloatTensor(x_train)#float
                train_pred_y = net(x_train)
                values, labels = torch.max(train_pred_y, 1)
                train_acc = accuracy_score(y_train,         # 計算訓練資料準確度
                                   labels)

                x_test = torch.FloatTensor(x_test)
                valid_pred_y = net(x_test)
                values, labels = torch.max(valid_pred_y, 1)
                valid_acc = accuracy_score(y_test,         # 計算驗證資料準確度
                                   labels)

                train_acc_list.append(train_acc)
                valid_acc_list.append(valid_acc)

Epoch 1
Epoch 6
Epoch 11
Epoch 16
Epoch 21
Epoch 26
Epoch 31
Epoch 36
Epoch 41
Epoch 46
Epoch 1
Epoch 6
Epoch 11
Epoch 16
Epoch 21
Epoch 26
Epoch 31
Epoch 36
Epoch 41
Epoch 46
Epoch 1
Epoch 6
Epoch 11
Epoch 16
Epoch 21
Epoch 26
Epoch 31
Epoch 36
Epoch 41
Epoch 46
Epoch 1
Epoch 6
Epoch 11
Epoch 16
Epoch 21
Epoch 26
Epoch 31
Epoch 36
Epoch 41
Epoch 46
Epoch 1
Epoch 6
Epoch 11
Epoch 16
Epoch 21
Epoch 26
Epoch 31
Epoch 36
Epoch 41
Epoch 46


In [21]:
print((
    'average train accuracy: {}\n' +
    'min train accuracy: {}\n' +
    'max train accuracy: {}').format(
    np.mean(train_acc_list),                          # 輸出平均訓練準確度
    np.min(train_acc_list),                           # 輸出最低訓練準確度
    np.max(train_acc_list)                            # 輸出最高訓練準確度
))
print()
print((
    'average valid accuracy: {}\n' +
    'min valid accuracy: {}\n' +
    'max valid accuracy: {}').format(
    np.mean(valid_acc_list),                          # 輸出平均驗證準確度
    np.min(valid_acc_list),                           # 輸出最低驗證準確度
    np.max(valid_acc_list)                            # 輸出最高驗證準確度
))

average train accuracy: 0.5190058639329465
min train accuracy: 0.4734806629834254
max train accuracy: 0.5582551076753175

average valid accuracy: 0.5190593432616817
min valid accuracy: 0.4370860927152318
max valid accuracy: 0.5938189845474614


In [22]:
test_var = Variable(torch.FloatTensor(x_test), requires_grad=True)
#test_y = Variable(torch.LongTensor(y_val), requires_grad=True)
with torch.no_grad(): #深度學習模型在訓練時會自動計算梯度，若於分析模型在目標函數的表現時不想花多餘資源計算梯度可以使用 with to.no_grad():
    result = net(test_var)

values, labels = torch.max(result, 1) # 0以行取max / 0以列取max

#MAX會回傳誰比較大 跟該index
num_right = np.sum(labels.data.numpy() == y_test)
#ACC = 猜中幾個
print('Accuracy {:.8f}'.format(num_right / len(y_test)))

Accuracy 0.53761062


比rr直接訓練有好一點（47-> 53.761062）

### 比較三個模型
* RL: 0.47808765
* SVM: 0.53761062
* RR: 0.53761062

其中以做過crossvalidation的RR最好。

## 試試看別的dataset
以hw2的鐵達尼號資料舉例

## RR

[參考這個網站](https://www.kaggle.com/kiranscaria/titanic-pytorch)
###  資料前處理（不重要）

In [23]:
dataset = pd.read_csv('./train_t.csv') #用來當training資料的
X_test = pd.read_csv('./test_t.csv')

dataset_title = [i.split(',')[1].split('.')[0].strip() for i in dataset['Name']]
dataset['Title'] = pd.Series(dataset_title)
dataset.tail(5)
dataset['Title'].value_counts()
dataset['Title'] = dataset['Title'].replace(['Lady', 'the Countess', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona', 'Ms', 'Mme', 'Mlle'], 'Rare')

dataset_title = [i.split(',')[1].split('.')[0].strip() for i in X_test['Name']]
X_test['Title'] = pd.Series(dataset_title)
X_test['Title'].value_counts()
X_test['Title'] = X_test['Title'].replace(['Lady', 'the Countess', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona', 'Ms', 'Mme', 'Mlle'], 'Rare')

dataset['FamilyS'] = dataset['SibSp'] + dataset['Parch'] + 1
X_test['FamilyS'] = X_test['SibSp'] + X_test['Parch'] + 1

def family(x):
    if x < 2:
        return 'Single'
    elif x == 2:
        return 'Couple'
    elif x <= 4:
        return 'InterM'
    else:
        return 'Large'
    
dataset['FamilyS'] = dataset['FamilyS'].apply(family)
X_test['FamilyS'] = X_test['FamilyS'].apply(family)
dataset['Embarked'].fillna(dataset['Embarked'].mode()[0], inplace=True)
X_test['Embarked'].fillna(X_test['Embarked'].mode()[0], inplace=True)
dataset['Age'].fillna(dataset['Age'].median(), inplace=True)
X_test['Age'].fillna(X_test['Age'].median(), inplace=True)
X_test['Fare'].fillna(X_test['Fare'].median(), inplace=True)

#dataset捨棄不用的欄位
dataset = dataset.drop(['PassengerId', 'Cabin', 'Name', 'SibSp', 'Parch', 'Ticket'], axis=1)
X_test_passengers = X_test['PassengerId']
X_test = X_test.drop(['PassengerId', 'Cabin', 'Name', 'SibSp', 'Parch', 'Ticket'], axis=1)

X_train = dataset.iloc[:, 1:8] #x捨棄Survived , .value的話會捨棄掉pclass那些

Y_train = dataset.iloc[:, 0].values #y是survived

# 註：test只保留特徵值，捨棄特徵名字

## 如果特徵中有string，將它轉換成數字

In [24]:
# Converting categorical values to one-hot representation 用pandes

X_train = pd.get_dummies(data=X_train, columns=['Sex', 'Embarked', 'Title', 'FamilyS'])
X_test = pd.get_dummies(data=X_test, columns=['Sex', 'Embarked', 'Title', 'FamilyS'])

In [25]:
# 所有運用到的資料全部變成value

X_train = X_train.values
X_test = X_test.values

### 開始訓練！

In [26]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(17, 200) #270自己設定的嘛0.0
        self.fc2 = nn.Linear(200, 2)
        
    def forward(self, x):
        x = self.fc1(x)
        x = F.dropout(x, p=0.1)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.sigmoid(x)
        
        return x
    
net = Net()

batch_size = 50 # 一組50（一次訓練訓練個 TRAIN DATA / 50次）
num_epochs = 50 # 訓練50次
learning_rate = 0.01

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

train_acc_list = []                                   # 儲存每次訓練模型的準確度
valid_acc_list = []                                   # 儲存每次驗證模型的準確度

# 切train val
kf = KFold(n_splits=5, shuffle=True)
kf.get_n_splits(X_train)#因為x y數量一樣，所以把一個拿進去切就好

for train_index, test_index in kf.split(X_train):
    #print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = X_train[train_index], X_train[test_index]
    y_train, y_test = Y_train[train_index], Y_train[test_index]
    batch_no = len(x_train) // batch_size
    for epoch in range(num_epochs):
        if epoch % 5 == 0:
            print('Epoch {}'.format(epoch+1))

        # 每個 batch learning
        for i in range(batch_no):
            #print('batch no:', i)
            start = i * batch_size
            end = start + batch_size
            
            net.train()# 進入非訓練模式
            
            #丟給model必須是Variable  才能backward
            x_var = torch.FloatTensor(x_train[start:end])#float
            y_var = torch.LongTensor(y_train[start:end])#int

            # Forward + Backward + Optimize
            optimizer.zero_grad()
            ypred_var = net(x_var) 
            loss =criterion(ypred_var, y_var)
            loss.backward()
            optimizer.step()
            
            net.eval() # 進入非訓練模式
            with torch.no_grad():
                
                x_train = torch.FloatTensor(x_train)#float
                train_pred_y = net(x_train)
                values, labels = torch.max(train_pred_y, 1)
                train_acc = accuracy_score(y_train,         # 計算訓練資料準確度
                                   labels)

                x_test = torch.FloatTensor(x_test)
                valid_pred_y = net(x_test)
                values, labels = torch.max(valid_pred_y, 1)
                valid_acc = accuracy_score(y_test,         # 計算驗證資料準確度
                                   labels)

                train_acc_list.append(train_acc)
                valid_acc_list.append(valid_acc)

Epoch 1
Epoch 6
Epoch 11
Epoch 16
Epoch 21
Epoch 26
Epoch 31
Epoch 36
Epoch 41
Epoch 46
Epoch 1
Epoch 6
Epoch 11
Epoch 16
Epoch 21
Epoch 26
Epoch 31
Epoch 36
Epoch 41
Epoch 46
Epoch 1
Epoch 6
Epoch 11
Epoch 16
Epoch 21
Epoch 26
Epoch 31
Epoch 36
Epoch 41
Epoch 46
Epoch 1
Epoch 6
Epoch 11
Epoch 16
Epoch 21
Epoch 26
Epoch 31
Epoch 36
Epoch 41
Epoch 46
Epoch 1
Epoch 6
Epoch 11
Epoch 16
Epoch 21
Epoch 26
Epoch 31
Epoch 36
Epoch 41
Epoch 46


In [27]:
print((
    'average train accuracy: {}\n' +
    'min train accuracy: {}\n' +
    'max train accuracy: {}').format(
    np.mean(train_acc_list),                          # 輸出平均訓練準確度
    np.min(train_acc_list),                           # 輸出最低訓練準確度
    np.max(train_acc_list)                            # 輸出最高訓練準確度
))
print()
print((
    'average valid accuracy: {}\n' +
    'min valid accuracy: {}\n' +
    'max valid accuracy: {}').format(
    np.mean(valid_acc_list),                          # 輸出平均驗證準確度
    np.min(valid_acc_list),                           # 輸出最低驗證準確度
    np.max(valid_acc_list)                            # 輸出最高驗證準確度
))

average train accuracy: 0.8387749809207135
min train accuracy: 0.6179775280898876
max train accuracy: 0.8681626928471248

average valid accuracy: 0.8182128733735664
min valid accuracy: 0.6033519553072626
max valid accuracy: 0.888268156424581


## 討論：為什麼RR 最好
其實這樣的資料太難預測，什麼模型都無法有突破性的展現。
rr比較好我猜想是因為我加入了crossvalidation的關係。
所以股票投資有風險，小心為妙！